# Part II: SDAA-based Sequential Group Recommendations (25 points)

**Students:** Oskari Perikangas, Xiaosi Huang  
**Date:** November 10, 2025


---
# Step 1: Design Plan (7 points)
SDAA (Satisfaction and Disagreement Aware Aggregation)

**State:**
- Satisfaction history for each user: `sat_history[u] = [sat_round1, sat_round2, ...]`
- Group disagreement trend over previous rounds
- Current recommendation round index `j`
- Total number of rounds: `μ` (mu)
- User rating profiles for initial alpha calculation

**Formulas:**
$$satO(u, RS) = \frac{\sum_{j=1}^{\mu} sat(u, Gr_j)}{\mu}$$
$$groupDisO(RS) = \max_{u \in G} satO(u, RS) - \min_{u \in G} satO(u, RS)$$

---
**Action: SDAA Dynamic Aggregation**

- Use SDAA method with practical adaptation:
  $$score(G,i,j) = (1 - \alpha_j) \times avg\_score(G,i,j) + \alpha_j \times least\_score(G,i,j)$$
  
  Where:
  - $avg\_score(G,i,j) = \frac{\sum_{u \in G} p_j(u,i)}{|G|}$ (Average method)
  - $least\_score(G,i,j) = \min_{u \in G} p_j(u,i)$ (Least Misery method)

- **α_j Calculation**:
  - **For round 1**: $\alpha_1 = 0.1 + 0.5 \times \sigma(\text{user\_ratings})$
    - Uses standard deviation of user average historical ratings
    - Maps to range [0.1, 0.6] based on user rating diversity
  - **For rounds j > 1**: $\alpha_j = \max_{u \in G} sat(u,Gr_{j-1}) - \min_{u \in G} sat(u,Gr_{j-1})$

---
**Reward:**
- Use `R_sd` reward function that balances satisfaction and disagreement:

$$R_{sd}(RS^j) = 2 \times \frac{groupSatO(RS^j) \times (1 - groupDisO(RS^j))}{groupSatO(RS^j) + (1 - groupDisO(RS^j))}$$

---
#### ALGORITHM WORKFLOW:

1. **Initialize**: 
   - Empty satisfaction history for all users
   - Calculate initial α₁ based on user rating behavior diversity

2. **For each round `j = 1` to `μ`:**
   - **If j = 1**: Use calculated initial α based on user diversity
   - **If j > 1**: Calculate α_j = max_u sat(u,Gr_{j-1}) - min_u sat(u,Gr_{j-1})
   - Generate individual predictions: `p_j(u,i)` for all users and items
   - Compute group scores: `score(G,i,j) = (1 - α_j) × avg + α_j × least`
   - Select top-k items as group recommendation `Gr_j`
   - Calculate round satisfaction: `sat(u,Gr_j)` for each user
   - Update state: Append satisfaction scores to history
   - Compute reward: `R_sd(RS^j)` for performance tracking

3. **Output**: Complete recommendation sequence `RS = {Gr_1, Gr_2, ..., Gr_μ}`

#### SATISFACTION CALCULATION:
$$sat(u, Gr_j) = \frac{\text{avg}(p_j(u, i \in Gr_j)) - 1}{4}$$

**Implementation Approach:**
- Normalizes predicted ratings from 1-5 scale to 0-1 satisfaction scale
- Provides meaningful satisfaction values for effective SDAA adjustment
- Ensures practical computation within reasonable bounds

---

# Step 2: Explanations and clarifications (6 points)
#### Effectiveness evaluation of sequential recommendation methods:

**Dynamic Adaptation Demonstrated:**
- **Round 1**: Initial α=0.2765 based on user rating diversity
- **Round 2**: α=0.4216 based on actual satisfaction disparity from Round 1
- **Round 3**: α=0.4415 further adjusted based on Round 2 satisfaction

**Fairness Enforcement:**
- Increasing α values (0.2765 → 0.4216 → 0.4415) show stronger emphasis on Least Misery
- Protects User 599 (lowest satisfaction) while maintaining reasonable group satisfaction

**Performance Metrics:**
- Final group satisfaction: 0.7285
- Final group disagreement: 0.4170  
- Average reward across rounds: 0.6486
- Balanced performance between satisfaction and fairness

**Advantages over Static Methods:**
- Adapts to evolving group dynamics across multiple rounds
- Automatically balances between Average and Least Misery based on actual user satisfaction
- Provides theoretical foundation with practical implementation
- Addresses cold-start problem through informed initial α calculation

# Step 3: Implementation (7 points)

In [1]:
import pandas as pd
import numpy as np
from collections import defaultdict
import warnings
warnings.filterwarnings('ignore')

print("=== Loading and Preparing MovieLens Dataset ===")

# Load ratings data
ratings_df = pd.read_csv("../data/ml-latest-small/ratings.csv")

# Standardize column names
if 'userId' in ratings_df.columns:
    ratings_df.rename(columns={'userId': 'user_id', 'movieId': 'item_id'}, inplace=True)

# Create user_ratings_dict for efficient access
user_ratings_dict = {}
for user_id in ratings_df['user_id'].unique():
    user_ratings_dict[user_id] = {}
    user_data = ratings_df[ratings_df['user_id'] == user_id]
    for _, row in user_data.iterrows():
        user_ratings_dict[user_id][row['item_id']] = row['rating']

print(f"Dataset loaded: {len(user_ratings_dict)} users, {len(ratings_df)} ratings")
print(f"Sample - User 1 rated {len(user_ratings_dict[1])} movies")

=== Loading and Preparing MovieLens Dataset ===
Dataset loaded: 610 users, 100836 ratings
Sample - User 1 rated 232 movies


In [2]:
def compare_popularity_thresholds():
    """Compare different popularity thresholds for movie selection"""
    movie_counts = ratings_df.groupby('item_id').size()
    
    # Analyze different popularity thresholds
    thresholds = [20, 30, 50, 75, 100]
    for threshold in thresholds:
        count = len(movie_counts[movie_counts >= threshold])
        percentage = (count / len(movie_counts)) * 100
        print(f"min_ratings={threshold}: {count} movies ({percentage:.1f}% of total)")
    
    # Analyze rating coverage for our test users
    test_group = [1, 414, 599]
    for user_id in test_group:
        user_rated = set(ratings_df[ratings_df['user_id'] == user_id]['item_id'])
        popular_30 = set(movie_counts[movie_counts >= 30].index)
        popular_50 = set(movie_counts[movie_counts >= 50].index)
        
        overlap_30 = len(user_rated & popular_30)
        overlap_50 = len(user_rated & popular_50)
        
        print(f"User {user_id}: {len(user_rated)} rated movies, "
              f"{overlap_30} overlap with threshold=30 ({overlap_30/len(user_rated)*100:.1f}%), "
              f"{overlap_50} overlap with threshold=50 ({overlap_50/len(user_rated)*100:.1f}%)")

# Call the comparison function
compare_popularity_thresholds()

min_ratings=20: 1297 movies (13.3% of total)
min_ratings=30: 882 movies (9.1% of total)
min_ratings=50: 450 movies (4.6% of total)
min_ratings=75: 236 movies (2.4% of total)
min_ratings=100: 138 movies (1.4% of total)
User 1: 232 rated movies, 158 overlap with threshold=30 (68.1%), 117 overlap with threshold=50 (50.4%)
User 414: 2698 rated movies, 809 overlap with threshold=30 (30.0%), 429 overlap with threshold=50 (15.9%)
User 599: 2478 rated movies, 703 overlap with threshold=30 (28.4%), 388 overlap with threshold=50 (15.7%)


In [3]:
class SequentialGroupState:
    """
    Manages state for sequential group recommendations according to SQUIRREL framework
    Tracks satisfaction history and calculates all required parameters
    """
    
    def __init__(self, group_users, user_ratings_dict):
        self.group_users = group_users
        self.user_ratings_dict = user_ratings_dict  # Access to user rating data
        self.sat_history = {user: [] for user in group_users}
        self.recommendation_history = []
        self.current_round = 0
        self.initial_alpha_calculated = False
        self.initial_alpha = 0.0
        
    def _calculate_initial_alpha_from_user_diversity(self):
        """
        Calculate initial alpha based on user rating behavior diversity
        Uses standard deviation of user average ratings as diversity indicator
        """
        user_avg_ratings = []
        
        for user_id in self.group_users:
            if user_id in self.user_ratings_dict:
                ratings = list(self.user_ratings_dict[user_id].values())
                if ratings:
                    user_avg_ratings.append(np.mean(ratings))
        
        if len(user_avg_ratings) < 2:
            return 0.3  # Default moderate alpha value
            
        # Calculate standard deviation of user average ratings as diversity metric
        rating_std = np.std(user_avg_ratings)
        
        # Map standard deviation to alpha range [0.1, 0.6]
        # Higher rating diversity results in higher initial alpha (more focus on fairness)
        initial_alpha = 0.1 + (rating_std / 2.0) * 0.5
        return float(max(0.1, min(0.6, initial_alpha)))
    
    def get_satisfaction_disparity(self):
        """
        Calculate α_j for SDAA method: α_j = max_u sat(u,Gr_{j-1}) - min_u sat(u,Gr_{j-1})
        """
        # For round 1: use initial alpha since no previous round data exists
        if self.current_round == 0:
            # Start of round 1 - calculate initial alpha based on user diversity
            if not self.initial_alpha_calculated:
                self.initial_alpha = self._calculate_initial_alpha_from_user_diversity()
                self.initial_alpha_calculated = True
            return self.initial_alpha
        
        # For rounds 2 and beyond: use previous round satisfaction data
        # When current_round = 1 (after round 1), calculate alpha for round 2
        # When current_round = 2 (after round 2), calculate alpha for round 3
        
        all_prev_sats = []
        for user in self.group_users:
            # Need at least current_round number of satisfaction entries
            # For calculating alpha at round j, the (j-1)th satisfaction values are required
            if len(self.sat_history[user]) >= self.current_round:
                # The previous round's satisfaction is at index current_round-1
                prev_sat = self.sat_history[user][self.current_round - 1]
                all_prev_sats.append(prev_sat)
        
        if not all_prev_sats or len(all_prev_sats) < len(self.group_users):
            return 0.0
            
        min_sat = min(all_prev_sats)
        max_sat = max(all_prev_sats)
        alpha = max_sat - min_sat
        
        return float(max(0.0, min(1.0, alpha)))
    
    def update_round(self, group_recommendations, user_satisfactions):
        """
        Update state after each recommendation round
        """
        self.current_round += 1
        self.recommendation_history.append(group_recommendations)
        
        for user in self.group_users:
            if user in user_satisfactions:
                self.sat_history[user].append(float(user_satisfactions[user]))
    
    def calculate_overall_satisfaction(self):
        """
        Calculate satO(u, RS) = average satisfaction across all rounds for each user
        satO(u, RS) = Σ_{j=1}^μ sat(u, Gr_j) / μ
        """
        satO = {}
        for user in self.group_users:
            if self.sat_history[user]:
                satO[user] = float(np.mean(self.sat_history[user]))
            else:
                satO[user] = 0.0
        return satO
    
    def calculate_group_satisfaction(self):
        """
        Calculate groupSatO(RS) = average overall satisfaction across all group members
        groupSatO(RS) = Σ_{u∈G} satO(u, RS) / |G|
        """
        satO = self.calculate_overall_satisfaction()
        if not satO:
            return 0.0
        return float(np.mean(list(satO.values())))
    
    def calculate_group_disagreement(self):
        """
        Calculate groupDisO(RS) = max_u satO(u, RS) - min_u satO(u, RS)
        """
        satO = self.calculate_overall_satisfaction()
        if not satO:
            return 0.0
        return float(max(satO.values()) - min(satO.values()))
    
    def get_previous_round_satisfactions(self):
        """
        Get satisfaction scores from the previous round
        """
        if self.current_round == 0:
            return {user: 0.0 for user in self.group_users}
        prev_satisfactions = {}
        for user in self.group_users:
            if len(self.sat_history[user]) >= self.current_round:
                prev_satisfactions[user] = self.sat_history[user][-1]
            else:
                prev_satisfactions[user] = 0.0
        return prev_satisfactions

In [4]:
class SQUIRRELRecommender:
    """
    Complete implementation of SQUIRREL framework with all methods
    """
    
    def __init__(self, group_users, ratings_df, user_ratings_dict):
        # Pass user_ratings_dict to state for initial alpha calculation
        self.state = SequentialGroupState(group_users, user_ratings_dict)
        self.ratings_df = ratings_df
        self.user_ratings_dict = user_ratings_dict
        self.popular_movies = self._get_popular_movies()
        self.used_movies = set()
    
    def _get_popular_movies(self, min_ratings=30):
        """Identify popular movies for reliable predictions"""
        movie_counts = self.ratings_df.groupby('item_id').size()
        popular = movie_counts[movie_counts >= min_ratings].index.tolist()
        print(f"Selected {len(popular)} popular movies (min_ratings={min_ratings})")
        return popular
    
    def _pearson_similarity(self, user1_ratings, user2_ratings, min_common=5):
        """Calculate Pearson correlation similarity"""
        common_items = set(user1_ratings.keys()) & set(user2_ratings.keys())
        
        if len(common_items) < min_common:
            return 0.0
        
        ratings1 = [user1_ratings[item] for item in common_items]
        ratings2 = [user2_ratings[item] for item in common_items]
        
        if len(ratings1) < 2:
            return 0.0
            
        mean1 = np.mean(ratings1)
        mean2 = np.mean(ratings2)
        
        numerator = sum((r1 - mean1) * (r2 - mean2) for r1, r2 in zip(ratings1, ratings2))
        denom1 = np.sqrt(sum((r1 - mean1) ** 2 for r1 in ratings1))
        denom2 = np.sqrt(sum((r2 - mean2) ** 2 for r2 in ratings2))
        
        if denom1 == 0 or denom2 == 0:
            return 0.0
            
        return float(numerator / (denom1 * denom2))
    
    def predict_user_rating(self, user_id, item_id, k=10):
        """Predict user rating using collaborative filtering"""
        if user_id not in self.user_ratings_dict:
            return 3.0
        
        target_ratings = self.user_ratings_dict[user_id]
        target_mean = np.mean(list(target_ratings.values()))
        
        item_ratings = self.ratings_df[self.ratings_df['item_id'] == item_id]
        if len(item_ratings) == 0:
            return target_mean
        
        similarities = []
        for _, row in item_ratings.iterrows():
            neighbor_id = row['user_id']
            if neighbor_id != user_id and neighbor_id in self.user_ratings_dict:
                sim = self._pearson_similarity(target_ratings, self.user_ratings_dict[neighbor_id])
                if sim > 0.1:
                    similarities.append((sim, neighbor_id, row['rating']))
        
        if similarities:
            similarities.sort(reverse=True)
            top_neighbors = similarities[:k]
            
            weighted_sum = 0.0
            similarity_sum = 0.0
            
            for sim, neighbor_id, neighbor_rating in top_neighbors:
                neighbor_ratings = self.user_ratings_dict[neighbor_id]
                neighbor_mean = np.mean(list(neighbor_ratings.values()))
                weighted_sum += sim * (neighbor_rating - neighbor_mean)
                similarity_sum += sim
            
            if similarity_sum > 0:
                predicted = target_mean + (weighted_sum / similarity_sum)
                return float(max(1.0, min(5.0, predicted)))
        
        item_mean = item_ratings['rating'].mean()
        return float(item_mean)
    
    def calculate_round_satisfaction(self, user_id, group_recommendations):
        """
        Calculate sat(u, Gr_j) with corrected implementation
        Instead of using ALL available items (which makes denominator too large),
        we normalize based on the maximum possible satisfaction
        """
        if user_id not in self.user_ratings_dict:
            return 0.0
        
        # Calculate predicted ratings for group recommendations
        group_predictions = []
        for item in group_recommendations:
            pred = self.predict_user_rating(user_id, item)
            group_predictions.append(pred)
        
        if not group_predictions:
            return 0.0
        
        # Numerator: average of predicted ratings for group recommendations
        numerator = np.mean(group_predictions)
        
        # Normalize from 1-5 scale to 0-1 scale
        satisfaction = (numerator - 1.0) / 4.0
        
        return float(max(0.0, min(1.0, satisfaction)))
    
    def calculate_user_disagreement(self, user_id, user_satisfactions):
        """
        Calculate userDis(u, G, Gr_j) = max_{u'∈G} sat(u', Gr_j) - sat(u, Gr_j)
        """
        max_sat = max(user_satisfactions.values())
        user_sat = user_satisfactions.get(user_id, 0.0)
        return max_sat - user_sat
    
    def generate_individual_predictions(self, k_candidates=100):
        """
        Generate p_j(u,i) predictions for all users and candidate items
        """
        user_predictions = {}
        
        for user_id in self.state.group_users:
            if user_id not in self.user_ratings_dict:
                continue
                
            # Find unrated popular movies
            user_rated = set(self.ratings_df[self.ratings_df['user_id'] == user_id]['item_id'])
            user_unrated = [item for item in self.popular_movies 
                          if item not in user_rated and item not in self.used_movies]
            
            if len(user_unrated) < k_candidates:
                user_unrated = [item for item in self.popular_movies if item not in user_rated]
            
            # Generate predictions
            predictions = {}
            for item in user_unrated[:k_candidates]:
                pred = self.predict_user_rating(user_id, item)
                predictions[item] = pred
            
            user_predictions[user_id] = predictions
        
        return user_predictions
    
    def sdaa_aggregation(self, user_predictions):
        """
        SDAA method: score(G,i,j) = (1 - α_j) * avg_score(G,i,j) + α_j * least_score(G,i,j)
        where α_j = max_u sat(u,Gr_{j-1}) - min_u sat(u,Gr_{j-1})
        """
        alpha = self.state.get_satisfaction_disparity()
        
        print(f"    SDAA alpha parameter: {alpha:.4f}")
        
        all_movies = set()
        for predictions in user_predictions.values():
            all_movies.update(predictions.keys())
        
        movie_scores = {}
        
        for movie in all_movies:
            user_scores = []
            for predictions in user_predictions.values():
                if movie in predictions:
                    user_scores.append(predictions[movie])
            
            if not user_scores:
                continue
                
            avg_score = np.mean(user_scores)
            least_score = min(user_scores)
            
            # SDAA formula
            movie_scores[movie] = (1 - alpha) * avg_score + alpha * least_score
        
        return movie_scores
    
    def siaa_aggregation(self, user_predictions, b=0.5):
        """
        SIAA method: score(G,i,j) = Σ_{u∈G} w_u,j * p_j(u,i)
        where w_u,j = 1 - b * (1 - satO(u, RS_{j-1})) + b * userDis(u, G, Gr_{j-1})
        """
        # Get previous round data
        prev_satisfactions = self.state.get_previous_round_satisfactions()
        
        # Calculate weights for each user
        user_weights = {}
        for user_id in self.state.group_users:
            # Get overall satisfaction up to previous round
            satO = 0.0
            if self.state.sat_history[user_id]:
                satO = np.mean(self.state.sat_history[user_id])
            
            # Calculate user disagreement from previous round
            userDis = self.calculate_user_disagreement(user_id, prev_satisfactions)
            
            # SIAA weight formula
            w_u = 1 - b * (1 - satO) + b * userDis
            user_weights[user_id] = max(0.0, w_u)
        
        print(f"    SIAA user weights: {user_weights}")
        
        # Normalize weights
        total_weight = sum(user_weights.values())
        if total_weight > 0:
            for user_id in user_weights:
                user_weights[user_id] /= total_weight
        
        all_movies = set()
        for predictions in user_predictions.values():
            all_movies.update(predictions.keys())
        
        movie_scores = {}
        
        for movie in all_movies:
            weighted_score = 0.0
            for user_id, predictions in user_predictions.items():
                if movie in predictions:
                    weight = user_weights.get(user_id, 0.0)
                    weighted_score += weight * predictions[movie]
            
            if weighted_score > 0:
                movie_scores[movie] = weighted_score
        
        return movie_scores
    
    def average_aggregation(self, user_predictions):
        """Standard Average aggregation method"""
        all_movies = set()
        for predictions in user_predictions.values():
            all_movies.update(predictions.keys())
        
        movie_scores = {}
        
        for movie in all_movies:
            user_scores = []
            for predictions in user_predictions.values():
                if movie in predictions:
                    user_scores.append(predictions[movie])
            
            if user_scores:
                movie_scores[movie] = np.mean(user_scores)
        
        return movie_scores
    
    def least_misery_aggregation(self, user_predictions):
        """Standard Least Misery aggregation method"""
        all_movies = set()
        for predictions in user_predictions.values():
            all_movies.update(predictions.keys())
        
        movie_scores = {}
        
        for movie in all_movies:
            user_scores = []
            for predictions in user_predictions.values():
                if movie in predictions:
                    user_scores.append(predictions[movie])
            
            if user_scores:
                movie_scores[movie] = min(user_scores)
        
        return movie_scores
    
    def calculate_reward_rs(self):
        """
        Calculate R_s reward function: focus on satisfaction only
        R_s(RS^j) = groupSatO(RS^j)
        """
        return self.state.calculate_group_satisfaction()
    
    def calculate_reward_rsd(self):
        """
        Calculate R_sd reward function: balance satisfaction and disagreement
        R_sd(RS^j) = 2 * (groupSatO(RS^j) * (1 - groupDisO(RS^j))) / 
                     (groupSatO(RS^j) + (1 - groupDisO(RS^j)))
        """
        groupSatO = self.state.calculate_group_satisfaction()
        groupDisO = self.state.calculate_group_disagreement()
        
        if groupSatO + (1 - groupDisO) == 0:
            return 0.0
        
        reward = 2 * (groupSatO * (1 - groupDisO)) / (groupSatO + (1 - groupDisO))
        return float(reward)
    
    def recommend_round(self, k=5, method='sdaa'):
        """
        Generate recommendations for one round using specified method
        """
        user_predictions = self.generate_individual_predictions()
        
        if not user_predictions:
            return [], {user: 0.0 for user in self.state.group_users}
        
        # Apply specified aggregation method
        if method == 'sdaa':
            movie_scores = self.sdaa_aggregation(user_predictions)
        elif method == 'siaa':
            movie_scores = self.siaa_aggregation(user_predictions)
        elif method == 'average':
            movie_scores = self.average_aggregation(user_predictions)
        elif method == 'least_misery':
            movie_scores = self.least_misery_aggregation(user_predictions)
        else:
            movie_scores = self.sdaa_aggregation(user_predictions)
        
        if not movie_scores:
            return [], {user: 0.0 for user in self.state.group_users}
        
        # Select top-k recommendations
        sorted_movies = sorted(movie_scores.items(), key=lambda x: x[1], reverse=True)
        available_movies = [(movie, score) for movie, score in sorted_movies 
                          if movie not in self.used_movies]
        
        if len(available_movies) < k:
            available_movies = sorted_movies
        
        group_recommendations = [movie for movie, score in available_movies[:k]]
        self.used_movies.update(group_recommendations)
        
        # Calculate satisfactions using corrected formula
        user_satisfactions = {}
        for user_id in self.state.group_users:
            sat = self.calculate_round_satisfaction(user_id, group_recommendations)
            user_satisfactions[user_id] = sat
        
        self.state.update_round(group_recommendations, user_satisfactions)
        
        return group_recommendations, user_satisfactions
    
    def run_sequential_recommendation(self, num_rounds=3, k=5, method='sdaa', reward_function='rsd'):
        """
        Run complete sequential recommendation process
        """
        print(f"Starting sequential group recommendation for {num_rounds} rounds")
        print(f"Group members: {self.state.group_users}")
        print(f"Method: {method.upper()}, Reward: R_{reward_function}")
        print("-" * 60)

        all_recommendations = []
        round_rewards = []

        for round_num in range(1, num_rounds + 1):
            print(f"Round {round_num}:")
            
            recommendations, satisfactions = self.recommend_round(k, method)
            all_recommendations.append(recommendations)
            
            # Calculate reward based on specified function
            if reward_function == 'rs':
                reward = self.calculate_reward_rs()
            else:
                reward = self.calculate_reward_rsd()
            
            round_rewards.append(reward)
            
            print(f"  Recommendations: {recommendations}")
            # Format output for better readability
            formatted_satisfactions = {user: f"{sat:.4f}" for user, sat in satisfactions.items()}
            print(f"  User satisfactions: {formatted_satisfactions}")
            print(f"  Round reward (R_{reward_function}): {reward:.4f}")
            
            if round_num < num_rounds:
                next_alpha = self.state.get_satisfaction_disparity()
                print(f"  Next round satisfaction disparity: {next_alpha:.4f}")
            print()
        
        # Final statistics
        final_satO = self.state.calculate_overall_satisfaction()
        final_groupSatO = self.state.calculate_group_satisfaction()
        final_groupDisO = self.state.calculate_group_disagreement()
        
        print("=" * 60)
        print("FINAL RESULTS:")
        # Format final output
        formatted_final_satO = {user: f"{sat:.4f}" for user, sat in final_satO.items()}
        print(f"Overall user satisfactions (satO): {formatted_final_satO}")
        print(f"Final group satisfaction (groupSatO): {final_groupSatO:.4f}")
        print(f"Final group disagreement (groupDisO): {final_groupDisO:.4f}")
        print(f"Average reward across rounds: {np.mean(round_rewards):.4f}")
        
        return all_recommendations, round_rewards

In [5]:
# Test the complete SQUIRREL framework
print("=== Testing Complete SQUIRREL Framework ===")

# Define test group
test_group = [1, 414, 599]

# Initialize the recommender system
recommender = SQUIRRELRecommender(
    group_users=test_group,
    ratings_df=ratings_df,
    user_ratings_dict=user_ratings_dict
)

# Run sequential recommendations with SDAA method
print("\n" + "="*70)
print("EXECUTING SEQUENTIAL GROUP RECOMMENDATIONS WITH SDAA")
print("="*70)

recommendations, rewards = recommender.run_sequential_recommendation(
    num_rounds=3, 
    k=5,
    method='sdaa',
    reward_function='rsd'
)

print("=== SDAA Recommendation Test Complete ===")
print(f"Total rounds executed: {len(recommendations)}")
print(f"Final average reward: {np.mean(rewards):.4f}")

=== Testing Complete SQUIRREL Framework ===
Selected 882 popular movies (min_ratings=30)

EXECUTING SEQUENTIAL GROUP RECOMMENDATIONS WITH SDAA
Starting sequential group recommendation for 3 rounds
Group members: [1, 414, 599]
Method: SDAA, Reward: R_rsd
------------------------------------------------------------
Round 1:
    SDAA alpha parameter: 0.2765
  Recommendations: [293, 318, 353, 16, 364]
  User satisfactions: {1: '0.9893', 414: '0.7399', 599: '0.5677'}
  Round reward (R_rsd): 0.6590
  Next round satisfaction disparity: 0.4216

Round 2:
    SDAA alpha parameter: 0.4216
  Recommendations: [180, 454, 17, 497, 508]
  User satisfactions: {1: '0.9292', 414: '0.6647', 599: '0.4876'}
  Round reward (R_rsd): 0.6391
  Next round satisfaction disparity: 0.4415

Round 3:
    SDAA alpha parameter: 0.4415
  Recommendations: [541, 529, 337, 252, 524]
  User satisfactions: {1: '0.9216', 414: '0.7228', 599: '0.5337'}
  Round reward (R_rsd): 0.6477

FINAL RESULTS:
Overall user satisfactions (s

# Step 4: Presentation (5 points)

In [7]:
print("\n" + "="*70)
print("CONCLUSION: SDAA Sequential Recommendation Successfully Implemented")
print("="*70)
print("✓ Dynamic alpha adaptation demonstrated: 0.2765 → 0.4216 → 0.4415")
print("✓ Balanced performance achieved: Group Satisfaction = 0.7285, Disagreement = 0.4170")
print("✓ SDAA effectively protected minority interests while maintaining group satisfaction")
print("✓ Framework ready for extended testing with different groups and parameters")
print("="*70)


CONCLUSION: SDAA Sequential Recommendation Successfully Implemented
✓ Dynamic alpha adaptation demonstrated: 0.2765 → 0.4216 → 0.4415
✓ Balanced performance achieved: Group Satisfaction = 0.7285, Disagreement = 0.4170
✓ SDAA effectively protected minority interests while maintaining group satisfaction
✓ Framework ready for extended testing with different groups and parameters
